# Lab 5: Spark + Iceberg Integration

## 🎯 **Learning Objectives:**
- Integrate Spark with Apache Iceberg
- Learn to read and write Iceberg tables from Spark
- Understand schema evolution with Spark
- Practice time travel queries
- Implement partitioning strategies

## 📚 **Key Concepts:**
1. **Iceberg Catalog**: Metadata management for Iceberg tables
2. **Spark Extensions**: Iceberg Spark extensions for integration
3. **Schema Evolution**: Add/modify columns without rewriting data
4. **Time Travel**: Query historical snapshots
5. **Partitioning**: Optimize queries with partitioning

## 🏗️ **Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Spark SQL     │───▶│  Iceberg Catalog │───▶│  Iceberg Tables │
│   (Processing)  │    │  (Metadata)       │    │  (Data Files)   │
└─────────────────┘    └──────────────────┘    └─────────────────┘
         │                        │                        │
         ▼                        ▼                        ▼
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│ Read/Write      │    │ Schema Evolution  │    │ Time Travel     │
│ Operations      │    │ • Add columns     │    │ • Snapshots     │
│                 │    │ • Modify types    │    │ • History       │
└─────────────────┘    └──────────────────┘    └─────────────────┘
```

## 📊 **Use Cases:**
- **Data Lakehouse**: Store large datasets with ACID guarantees
- **Schema Evolution**: Evolve schemas without downtime
- **Time Travel**: Query data at different points in time
- **Partitioning**: Optimize query performance
- **Integration**: Combine Spark processing with Iceberg storage


In [ ]:
# Install and Import Dependencies
# Note: Iceberg Spark runtime JAR needs to be available
# For production, download iceberg-spark-runtime-3.5_2.12-1.4.0.jar
# and place it in Spark's jars directory or use --jars option

%pip install pyspark findspark pandas numpy pyarrow

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os

print("✅ Dependencies installed and imported successfully!")
print("\n⚠️ Note: Iceberg Spark runtime JAR is required for full functionality")
print("   Download from: https://iceberg.apache.org/releases/")
print("   Or use Spark packages: --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.0")


In [ ]:
# Initialize Spark Session with Iceberg Support
# Note: In production, you need to add Iceberg Spark runtime JAR
# This example shows the configuration needed

warehouse_path = "/tmp/spark_warehouse/iceberg"
os.makedirs(warehouse_path, exist_ok=True)

# Build Spark session
builder = SparkSession.builder \
    .appName("SparkIcebergIntegration") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# Uncomment when Iceberg JAR is available:
# builder = builder.config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
# builder = builder.config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
# builder = builder.config("spark.sql.catalog.spark_catalog.type", "hive")
# builder = builder.config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
# builder = builder.config("spark.sql.catalog.local.type", "hadoop")
# builder = builder.config("spark.sql.catalog.local.warehouse", warehouse_path)

spark = builder.getOrCreate()

# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

print("🚀 Spark Session initialized!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔗 Master URL: {spark.sparkContext.master}")
print(f"💾 Warehouse Path: {warehouse_path}")
print("\n⚠️ Note: Full Iceberg functionality requires Iceberg Spark runtime JAR")


## Exercise 1: Create Sample Data and Setup

### 🎯 **Learning Objectives:**
- Create sample datasets for Iceberg integration
- Understand data preparation for Iceberg tables
- Set up Spark DataFrames for Iceberg operations


In [ ]:
# Create Sample Data for Iceberg Integration
print("📊 Creating sample datasets for Spark + Iceberg Integration Lab...")

# Sample Sales Data with timestamps for time travel
sales_data = []
products = ['Laptop', 'Phone', 'Tablet', 'Headphones', 'Camera', 'Monitor', 'Keyboard', 'Mouse']
customers = ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank', 'Grace', 'Henry']
categories = ['Electronics', 'Accessories', 'Computing']

base_date = datetime.now() - timedelta(days=365)

for i in range(1000):
    sale_date = base_date + timedelta(days=random.randint(0, 365))
    sales_data.append({
        'sale_id': f'SALE_{i+1:04d}',
        'customer_name': random.choice(customers),
        'product_name': random.choice(products),
        'category': random.choice(categories),
        'quantity': random.randint(1, 5),
        'unit_price': round(random.uniform(50, 2000), 2),
        'sale_date': sale_date.strftime('%Y-%m-%d'),
        'sale_timestamp': sale_date.isoformat(),
        'region': random.choice(['North', 'South', 'East', 'West', 'Central'])
    })

# Create Spark DataFrame
sales_df = spark.createDataFrame(sales_data)

# Add calculated columns
sales_df = sales_df.withColumn(
    "total_amount", 
    col("quantity") * col("unit_price")
).withColumn(
    "sale_year",
    year(to_date(col("sale_date"), "yyyy-MM-dd"))
).withColumn(
    "sale_month",
    month(to_date(col("sale_date"), "yyyy-MM-dd"))
)

print(f"✅ Sample data created:")
print(f"   📊 Sales records: {len(sales_data)}")
print("\n📋 DataFrame Schema:")
sales_df.printSchema()
print("\n📊 Sample Data:")
sales_df.show(5, truncate=False)


In [ ]:
# Write to Iceberg Table (Example Pattern)
print("📝 Exercise 2: Write to Iceberg Table")
print("=" * 60)

print("\n1️⃣ Create Iceberg table using Spark SQL (when Iceberg JAR is available):")
print("""
# Example SQL to create Iceberg table:
spark.sql('''
    CREATE TABLE local.db.sales_iceberg (
        sale_id STRING,
        customer_name STRING,
        product_name STRING,
        category STRING,
        quantity INT,
        unit_price DOUBLE,
        sale_date DATE,
        sale_timestamp TIMESTAMP,
        region STRING,
        total_amount DOUBLE,
        sale_year INT,
        sale_month INT
    )
    USING ICEBERG
    PARTITIONED BY (sale_year, sale_month)
    TBLPROPERTIES (
        'write.format.default' = 'parquet',
        'write.parquet.compression-codec' = 'snappy'
    )
''')
""")

print("\n2️⃣ Write DataFrame to Iceberg table (when Iceberg JAR is available):")
print("""
# Example: Write DataFrame to Iceberg
sales_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .option("path", "local.db.sales_iceberg") \
    .save()
""")

print("\n3️⃣ Append data to existing Iceberg table:")
print("""
# Example: Append new data
new_sales_df.write \
    .format("iceberg") \
    .mode("append") \
    .option("path", "local.db.sales_iceberg") \
    .save()
""")

print("\n4️⃣ Write with partitioning:")
print("""
# Example: Write with partitionBy
sales_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .partitionBy("sale_year", "sale_month") \
    .option("path", "local.db.sales_iceberg") \
    .save()
""")

print("\n⚠️ Note: These operations require Iceberg Spark runtime JAR")
print("   For now, we'll demonstrate the concepts with regular Parquet format")


In [ ]:
# Alternative: Write to Parquet (similar pattern, works without Iceberg JAR)
print("\n5️⃣ Write to Parquet format (similar to Iceberg pattern):")

parquet_path = f"{warehouse_path}/sales_parquet"
sales_df.write \
    .mode("overwrite") \
    .partitionBy("sale_year", "sale_month") \
    .parquet(parquet_path)

print(f"✅ Data written to: {parquet_path}")

# Read back to verify
parquet_df = spark.read.parquet(parquet_path)
print(f"\n📊 Records read back: {parquet_df.count()}")
print("\n📋 Sample data:")
parquet_df.show(5, truncate=False)


## Exercise 3: Read from Iceberg Table

### 🎯 **Learning Objectives:**
- Read data from Iceberg tables
- Understand query patterns
- Learn to filter partitioned data
- Practice reading specific snapshots


In [ ]:
# Read from Iceberg Table (Example Pattern)
print("📖 Exercise 3: Read from Iceberg Table")
print("=" * 60)

print("\n1️⃣ Read entire Iceberg table (when Iceberg JAR is available):")
print("""
# Example: Read from Iceberg table
iceberg_df = spark.read \
    .format("iceberg") \
    .load("local.db.sales_iceberg")

iceberg_df.show()
""")

print("\n2️⃣ Read with filters (partition pruning):")
print("""
# Example: Filter by partition columns (efficient)
iceberg_df = spark.read \
    .format("iceberg") \
    .load("local.db.sales_iceberg") \
    .filter("sale_year = 2024 AND sale_month = 1")

iceberg_df.show()
""")

print("\n3️⃣ Read using Spark SQL:")
print("""
# Example: Query Iceberg table with SQL
spark.sql('''
    SELECT 
        customer_name,
        product_name,
        SUM(total_amount) as total_spent
    FROM local.db.sales_iceberg
    WHERE sale_year = 2024
    GROUP BY customer_name, product_name
    ORDER BY total_spent DESC
''').show()
""")

print("\n⚠️ Note: These operations require Iceberg Spark runtime JAR")
print("   For now, we'll demonstrate with Parquet format")


In [ ]:
# Alternative: Read from Parquet (similar pattern)
print("\n4️⃣ Read from Parquet with partition pruning:")

# Read with partition filter (efficient)
filtered_df = spark.read.parquet(parquet_path) \
    .filter("sale_year = 2024 AND sale_month = 1")

print(f"📊 Filtered records: {filtered_df.count()}")
filtered_df.show(5, truncate=False)

# Read and aggregate
print("\n5️⃣ Read and aggregate (similar to Iceberg SQL):")
aggregated_df = spark.read.parquet(parquet_path) \
    .filter("sale_year = 2024") \
    .groupBy("customer_name", "product_name") \
    .agg(sum("total_amount").alias("total_spent")) \
    .orderBy(desc("total_spent"))

aggregated_df.show(10, truncate=False)


## Exercise 4: Schema Evolution

### 🎯 **Learning Objectives:**
- Understand Iceberg schema evolution capabilities
- Add new columns to existing tables
- Modify column types
- Practice schema changes without data rewrite


In [ ]:
# Schema Evolution (Example Pattern)
print("🔄 Exercise 4: Schema Evolution")
print("=" * 60)

print("\n1️⃣ Add new column to Iceberg table (when Iceberg JAR is available):")
print("""
# Example: Add new column using ALTER TABLE
spark.sql('''
    ALTER TABLE local.db.sales_iceberg
    ADD COLUMN discount_amount DOUBLE
''')
""")

print("\n2️⃣ Modify column type:")
print("""
# Example: Change column type
spark.sql('''
    ALTER TABLE local.db.sales_iceberg
    ALTER COLUMN unit_price TYPE DECIMAL(10, 2)
''')
""")

print("\n3️⃣ Rename column:")
print("""
# Example: Rename column
spark.sql('''
    ALTER TABLE local.db.sales_iceberg
    RENAME COLUMN sale_id TO transaction_id
''')
""")

print("\n4️⃣ Drop column:")
print("""
# Example: Drop column
spark.sql('''
    ALTER TABLE local.db.sales_iceberg
    DROP COLUMN sale_timestamp
''')
""")

print("\n5️⃣ Add data with new schema:")
print("""
# Example: Write DataFrame with new column
new_sales_df = sales_df.withColumn("discount_amount", lit(0.0))
new_sales_df.write \
    .format("iceberg") \
    .mode("append") \
    .option("path", "local.db.sales_iceberg") \
    .save()
""")

print("\n✅ Key Benefits of Iceberg Schema Evolution:")
print("   - Add columns without rewriting existing data")
print("   - Backward compatible: old queries still work")
print("   - Forward compatible: new queries work with old data")
print("   - No downtime required")


## Exercise 5: Time Travel Queries

### 🎯 **Learning Objectives:**
- Query historical snapshots of data
- Understand snapshot management
- Practice time travel queries
- Learn to rollback to previous versions


In [ ]:
# Time Travel Queries (Example Pattern)
print("⏰ Exercise 5: Time Travel Queries")
print("=" * 60)

print("\n1️⃣ Query table at specific snapshot (when Iceberg JAR is available):")
print("""
# Example: Query at snapshot ID
spark.sql('''
    SELECT *
    FROM local.db.sales_iceberg VERSION AS OF 12345
    WHERE sale_year = 2024
''').show()
""")

print("\n2️⃣ Query table at specific timestamp:")
print("""
# Example: Query at timestamp
spark.sql('''
    SELECT *
    FROM local.db.sales_iceberg TIMESTAMP AS OF '2024-01-15 10:00:00'
    WHERE sale_year = 2024
''').show()
""")

print("\n3️⃣ List all snapshots:")
print("""
# Example: View snapshot history
spark.sql('''
    SELECT *
    FROM local.db.sales_iceberg.snapshots
    ORDER BY committed_at DESC
''').show()
""")

print("\n4️⃣ Compare data between snapshots:")
print("""
# Example: Compare current vs previous snapshot
spark.sql('''
    SELECT 
        current.sale_id,
        current.total_amount as current_amount,
        previous.total_amount as previous_amount,
        current.total_amount - previous.total_amount as difference
    FROM local.db.sales_iceberg current
    FULL OUTER JOIN local.db.sales_iceberg VERSION AS OF 12345 previous
        ON current.sale_id = previous.sale_id
    WHERE current.total_amount != previous.total_amount
''').show()
""")

print("\n✅ Key Benefits of Time Travel:")
print("   - Query historical data at any point in time")
print("   - Audit trail of all changes")
print("   - Rollback to previous versions if needed")
print("   - Compare data across different time points")


## Exercise 6: Partitioning Strategies

### 🎯 **Learning Objectives:**
- Understand Iceberg partitioning
- Choose appropriate partitioning strategies
- Learn about hidden partitioning
- Practice partition evolution


In [ ]:
# Partitioning Strategies (Example Pattern)
print("📦 Exercise 6: Partitioning Strategies")
print("=" * 60)

print("\n1️⃣ Create table with identity partitioning:")
print("""
# Example: Partition by year and month
spark.sql('''
    CREATE TABLE local.db.sales_iceberg (
        sale_id STRING,
        customer_name STRING,
        product_name STRING,
        sale_date DATE,
        total_amount DOUBLE,
        sale_year INT,
        sale_month INT
    )
    USING ICEBERG
    PARTITIONED BY (sale_year, sale_month)
''')
""")

print("\n2️⃣ Create table with bucket partitioning:")
print("""
# Example: Bucket by customer_name
spark.sql('''
    CREATE TABLE local.db.sales_iceberg (
        sale_id STRING,
        customer_name STRING,
        product_name STRING,
        total_amount DOUBLE
    )
    USING ICEBERG
    PARTITIONED BY (bucket(10, customer_name))
''')
""")

print("\n3️⃣ Create table with transform partitioning:")
print("""
# Example: Partition by date truncation
spark.sql('''
    CREATE TABLE local.db.sales_iceberg (
        sale_id STRING,
        customer_name STRING,
        sale_timestamp TIMESTAMP,
        total_amount DOUBLE
    )
    USING ICEBERG
    PARTITIONED BY (days(sale_timestamp))
''')
""")

print("\n4️⃣ Evolve partitioning (change partition spec):")
print("""
# Example: Change partitioning strategy
spark.sql('''
    ALTER TABLE local.db.sales_iceberg
    SET PARTITION SPEC (
        sale_year,
        sale_month,
        region
    )
''')
""")

print("\n✅ Key Benefits of Iceberg Partitioning:")
print("   - Hidden partitioning: partition values computed automatically")
print("   - Partition evolution: change partitioning without rewriting data")
print("   - Multiple partition strategies: identity, bucket, transform")
print("   - Query optimization: automatic partition pruning")


## Exercise 7: Integration Best Practices

### 🎯 **Learning Objectives:**
- Understand best practices for Spark + Iceberg integration
- Learn performance optimization techniques
- Practice error handling
- Understand monitoring and maintenance


In [ ]:
# Integration Best Practices
print("⭐ Exercise 7: Integration Best Practices")
print("=" * 60)

print("\n1️⃣ Optimize write operations:")
print("""
# Example: Use appropriate write mode and options
sales_df.write \
    .format("iceberg") \
    .mode("append") \
    .option("write-target-file-size-bytes", 536870912)  # 512MB
    .option("write.parquet.compression-codec", "snappy") \
    .option("write.parquet.row-group-size-bytes", 134217728)  # 128MB
    .partitionBy("sale_year", "sale_month") \
    .save("local.db.sales_iceberg")
""")

print("\n2️⃣ Optimize read operations:")
print("""
# Example: Use partition pruning and column projection
spark.read \
    .format("iceberg") \
    .load("local.db.sales_iceberg") \
    .filter("sale_year = 2024 AND sale_month = 1") \
    .select("sale_id", "customer_name", "total_amount") \
    .show()
""")

print("\n3️⃣ Maintain table metadata:")
print("""
# Example: Expire old snapshots
spark.sql('''
    CALL local.system.expire_snapshots(
        table => 'local.db.sales_iceberg',
        older_than => TIMESTAMP '2024-01-01 00:00:00',
        retain_last => 10
    )
''')
""")

print("\n4️⃣ Compact small files:")
print("""
# Example: Rewrite data files to optimize size
spark.sql('''
    CALL local.system.rewrite_data_files(
        table => 'local.db.sales_iceberg',
        strategy => 'binpack'
    )
''')
""")

print("\n✅ Best Practices Summary:")
print("   - Use appropriate partitioning for query patterns")
print("   - Optimize file sizes (aim for 512MB-1GB per file)")
print("   - Use column projection to reduce I/O")
print("   - Regularly expire old snapshots")
print("   - Compact small files periodically")
print("   - Monitor table metadata size")
print("   - Use appropriate compression codec")


## Summary

### ✅ What we learned:
1. **Spark + Iceberg Setup**: Configure Spark session with Iceberg extensions
2. **Write Operations**: Write DataFrames to Iceberg tables with partitioning
3. **Read Operations**: Read from Iceberg tables with filters and aggregations
4. **Schema Evolution**: Add/modify columns without rewriting data
5. **Time Travel**: Query historical snapshots of data
6. **Partitioning**: Implement efficient partitioning strategies
7. **Best Practices**: Optimize performance and maintain tables

### 🎯 Key Takeaways:
- Iceberg provides ACID transactions for data lakehouse
- Schema evolution allows changes without downtime
- Time travel enables querying historical data
- Partitioning strategies optimize query performance
- Integration with Spark enables scalable data processing

### 🚀 Next Steps:
- Install Iceberg Spark runtime JAR for full functionality
- Practice with larger datasets
- Explore advanced features (merge, delete, update)
- Integrate with other tools (dbt, Great Expectations)
- Deploy to production with proper monitoring

### 📚 Resources:
- [Apache Iceberg Documentation](https://iceberg.apache.org/)
- [Iceberg Spark Integration](https://iceberg.apache.org/docs/latest/spark-configuration/)
- [Iceberg Spark Runtime](https://iceberg.apache.org/releases/)
